# Interfacing *torch* to *heyoka.py*

```{note}
For an introduction on neural networks in *heyoka.py*, check out the tutorial: [Feed-Forward Neural Networks](<./ffnn.ipynb>).
```


```{warning}
This tutorial assumes [torch](https://pytorch.org/) is installed
```

*heyoka.py* is not a library meant for machine learning, nor it aspires to be one. However, given its support for feed-forward neural networks and their potential use in numerical integration, it might be useful to have a way to construct a `ffnn` from a torch model. This tutorial tackles this!


In [1]:
import torch
from torch import nn
#we use float64 as heyoka
torch.set_default_dtype(torch.float64)

class torch_net(nn.Module):
    def __init__(
        self,
        input_dim=4,
        hidden_layer_dims=[32, 32],
        output_dim=1,
        activation=nn.Tanh(),
    ):
        super(torch_net, self).__init__()

        dims = [input_dim] + hidden_layer_dims

        self.fcs = nn.ModuleList(
            [nn.Linear(dims[i], dims[i + 1]) for i in range(len(dims) - 1)]
        )

        self.act = activation
        self.acts = nn.ModuleList([self.act for _ in range(len(dims) - 1)])
        self.fc_out = nn.Linear(dims[-1], output_dim)

    def forward(self, x):
        for fc, act in zip(self.fcs, self.acts):
            x = act(fc(x))
        return self.act(self.fc_out(x))


Now let's write a function that takes the `torch` weights and biases, and returns them in a format that is compatible with `heyoka.ffnn`

In [2]:
import numpy as np
def weights_and_biases_heyoka(model):
    weights = {}
    biases = {}

    for name, param in model.named_parameters():
        if 'weight' in name:
            weights[name] = param.data.clone()
        elif 'bias' in name:
            biases[name] = param.data.clone()
    biases_torch=[]
    weights_torch=[]
    for idx in range(len(weights)):
        weights_torch.append(weights[list(weights.keys())[idx]].numpy())
        biases_torch.append(biases[list(biases.keys())[idx]].numpy())
        
    w_flat=[]
    b_flat=[]
    for i in range(len(weights_torch)):
        w_flat+=list(weights_torch[i].flatten())
        b_flat+=list(biases_torch[i].flatten())
    w_flat=np.array(w_flat)
    b_flat=np.array(b_flat)
    print(w_flat.shape)
    return np.concatenate((w_flat, b_flat))

We now instantiate the model and extract its weights and biases ready for constructing an `heyoka.ffnn` object

In [3]:
model = torch_net(input_dim=4, 
                  hidden_layer_dims=[32, 32],
                  output_dim=1,
                  activation=nn.Tanh())
flattened_weights = weights_and_biases_heyoka(model)

(1184,)


We instantiate the feed forward neural network in *heyoka.py* using those parameters:

In [4]:
import heyoka as hk

inp_1, inp_2, inp_3, inp_4=hk.make_vars("inp_1","inp_2","inp_3","inp_4")
model_heyoka=hk.model.ffnn(inputs=[inp_1, inp_2, inp_3, inp_4], 
                           nn_hidden=[32,32], 
                           n_out=1,
                           activations=[hk.tanh,hk.tanh,hk.tanh], 
                           nn_wb=flattened_weights)
model_heyoka_compiled=hk.make_cfunc(model_heyoka)

Good! How do we now verify the output is the same at inference? Let's generate some random inputs

In [5]:
random_input=torch.rand((4,1000000))
random_input_torch=random_input.t()
random_input_numpy=random_input.numpy()
out_array=np.zeros((1,1000000))

Now, let's compare the output of `heyoka.ffnn` and `torch` to see if they are identical

In [6]:
model_heyoka_compiled(random_input_numpy,outputs=out_array)

array([[-0.25788307, -0.21975401, -0.26027468, ..., -0.2427392 ,
        -0.24344253, -0.18246564]])

In [7]:
model(random_input_torch)

tensor([[-0.2579],
        [-0.2198],
        [-0.2603],
        ...,
        [-0.2427],
        [-0.2434],
        [-0.1825]], grad_fn=<TanhBackward0>)

In this way we have managed to port the *torch* model in *heyoka.py*, reproducing the same results... 